In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/adware'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-02 12:41:46.544383: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 107 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(107, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(74, 64, 64, 3)

In [6]:
X_test.shape

(33, 64, 64, 3)

In [7]:
y_train.shape

(74, 10)

In [8]:
y_test.shape

(33, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-02 12:41:50.130025: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-02 12:41:54.298158: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-02 12:41:54.298187: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-02 12:41:54.298959: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 12:41:54.304393: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-02 12:41:54.3048

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(74, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([9, 8, 0, 3, 5, 2, 5, 7, 4, 2, 7, 0, 3, 3, 6, 6, 9, 9, 6, 5, 4, 2,
       9, 2, 5, 6, 5, 3, 2, 2, 7, 5, 3, 3, 4, 0, 0, 4, 6, 4, 7, 9, 1, 1,
       5, 1, 7, 4, 5, 3, 3, 3, 2, 8, 1, 2, 8, 9, 4, 0, 8, 4, 6, 3, 2, 1,
       7, 3, 1, 2, 4, 2, 0, 6])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)
#Malware_model.add(GlobalAveragePooling2D())

2023-05-02 12:41:54.484051: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 2s 331ms/step - loss: 0.6714 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5518 - val_loss: 0.4510 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5326
Epoch 2/100
3/3 [==============================] - 0s 61ms/step - loss: 0.4748 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5112 - val_loss: 0.3584 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5311
Epoch 3/100
3/3 [==============================] - 0s 52ms/step - loss: 0.4213 - f1_m: 0.0556 - precision_m: 0.1548 - recall_m: 0.0339 - auc: 0.4731 - val_loss: 0.3573 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4731
Epoch 4/100
3/3 [==============================] - 0s 53ms/step - loss: 0.3969 - f1_m: 0.0271 - precision_m: 0.0677 - recall_m: 0.0169 - auc: 0.5139 - val_loss: 0.3450 - val_f1_

Epoch 32/100
3/3 [==============================] - 0s 53ms/step - loss: 0.1760 - f1_m: 0.5184 - precision_m: 0.8701 - recall_m: 0.3904 - auc: 0.9501 - val_loss: 0.2417 - val_f1_m: 0.1351 - val_precision_m: 0.5000 - val_recall_m: 0.0781 - val_auc: 0.8722
Epoch 33/100
3/3 [==============================] - 0s 49ms/step - loss: 0.1717 - f1_m: 0.4903 - precision_m: 0.8691 - recall_m: 0.3482 - auc: 0.9580 - val_loss: 0.2372 - val_f1_m: 0.1351 - val_precision_m: 0.5000 - val_recall_m: 0.0781 - val_auc: 0.8741
Epoch 34/100
3/3 [==============================] - 0s 60ms/step - loss: 0.1534 - f1_m: 0.6710 - precision_m: 0.8917 - recall_m: 0.5404 - auc: 0.9720 - val_loss: 0.2428 - val_f1_m: 0.1316 - val_precision_m: 0.4167 - val_recall_m: 0.0781 - val_auc: 0.8899
Epoch 35/100
3/3 [==============================] - 0s 57ms/step - loss: 0.1715 - f1_m: 0.5508 - precision_m: 0.7790 - recall_m: 0.4357 - auc: 0.9485 - val_loss: 0.2464 - val_f1_m: 0.1351 - val_precision_m: 0.5000 - val_recall_m: 0.078

3/3 [==============================] - 0s 49ms/step - loss: 0.0724 - f1_m: 0.8323 - precision_m: 0.8474 - recall_m: 0.8180 - auc: 0.9917 - val_loss: 0.2337 - val_f1_m: 0.7157 - val_precision_m: 0.7895 - val_recall_m: 0.6719 - val_auc: 0.8982
Epoch 65/100
3/3 [==============================] - 0s 52ms/step - loss: 0.0554 - f1_m: 0.8787 - precision_m: 0.9083 - recall_m: 0.8536 - auc: 0.9960 - val_loss: 0.2211 - val_f1_m: 0.7407 - val_precision_m: 0.7955 - val_recall_m: 0.7031 - val_auc: 0.8945
Epoch 66/100
3/3 [==============================] - 0s 51ms/step - loss: 0.0538 - f1_m: 0.9171 - precision_m: 0.9213 - recall_m: 0.9130 - auc: 0.9957 - val_loss: 0.2258 - val_f1_m: 0.7453 - val_precision_m: 0.8095 - val_recall_m: 0.7031 - val_auc: 0.8922
Epoch 67/100
3/3 [==============================] - 0s 52ms/step - loss: 0.0717 - f1_m: 0.8827 - precision_m: 0.9237 - recall_m: 0.8466 - auc: 0.9953 - val_loss: 0.2533 - val_f1_m: 0.7157 - val_precision_m: 0.7895 - val_recall_m: 0.6719 - val_auc: 

3/3 [==============================] - 0s 49ms/step - loss: 0.0378 - f1_m: 0.8922 - precision_m: 0.8965 - recall_m: 0.8880 - auc: 0.9959 - val_loss: 0.2742 - val_f1_m: 0.6964 - val_precision_m: 0.7292 - val_recall_m: 0.6719 - val_auc: 0.8870
Epoch 97/100
3/3 [==============================] - 0s 51ms/step - loss: 0.0450 - f1_m: 0.8880 - precision_m: 0.8880 - recall_m: 0.8880 - auc: 0.9947 - val_loss: 0.2723 - val_f1_m: 0.7264 - val_precision_m: 0.7857 - val_recall_m: 0.6875 - val_auc: 0.8888
Epoch 98/100
3/3 [==============================] - 0s 53ms/step - loss: 0.0440 - f1_m: 0.8740 - precision_m: 0.8780 - recall_m: 0.8701 - auc: 0.9952 - val_loss: 0.2709 - val_f1_m: 0.7364 - val_precision_m: 0.7826 - val_recall_m: 0.7031 - val_auc: 0.8791
Epoch 99/100
3/3 [==============================] - 0s 50ms/step - loss: 0.0333 - f1_m: 0.9497 - precision_m: 0.9624 - recall_m: 0.9375 - auc: 0.9977 - val_loss: 0.2758 - val_f1_m: 0.7321 - val_precision_m: 0.7708 - val_recall_m: 0.7031 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.2832 - f1_m: 0.7281 - precision_m: 0.7600 - recall_m: 0.7031 - auc: 0.8670


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.2832 - f1_m: 0.7281 - precision_m: 0.7600 - recall_m: 0.7031 - auc: 0.8670


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.7280700206756592


In [25]:
# Confusion Matrix
import numpy as np
import pandas as pd

In [26]:
y_pred = np.argmax(Malware_model.predict(X_test), axis=-1)

In [27]:
y_pred

array([0, 0, 7, 0, 2, 1, 0, 3, 3, 6, 0, 4, 7, 6, 3, 2, 5, 6, 6, 5, 4, 7,
       0, 3, 2, 6, 5, 1, 5, 1, 6, 3, 7])

In [28]:
y_test2 = np.argmax(y_test, axis=1)

In [29]:
y_test2

array([4, 5, 7, 0, 3, 1, 0, 2, 2, 6, 5, 4, 7, 3, 2, 3, 4, 9, 6, 1, 4, 7,
       0, 9, 3, 6, 5, 1, 9, 1, 2, 0, 7])

In [30]:
from sklearn import metrics
c_matrix = metrics.confusion_matrix(y_test2, y_pred)

In [31]:
import seaborn as sns
def confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [33]:
#class_names= batches.class_indices.keys()
#confusion_matrix(c_matrix, class_names, figsize = (20,7), fontsize=20)

NameError: name 'history' is not defined